# Introduction to the Keras Tuner

## Overview

Keras Tuner is a library that helps to pick the optimal set of hyperparameters for TensorFlow program. The process of selecting the right set of hyperparameters for machine learning (ML) application is called *hyperparameter tuning* or *hypertuning*.

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of ML program. Hyperparameters are of two types:

1. **Model hyperparameters** which influence model selection such as the number and width of hidden layers
2. **Algorithm hyperparameters** which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

## Setup

In [1]:
import tensorflow as tf
from tensorflow import keras

Install and import the Keras Tuner.

In [2]:
!pip install -q -U keras-tuner

In [3]:
import keras_tuner as kt

## Download and prepare the dataset

Use the Keras Tuner to find the best hyperparameters for a machine learning model that classifies images of clothing from the [Fashion MNIST dataset]

Load the data.

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Define the model

When building a model for hypertuning, the defination of the hyperparameter search space in addition to the model architecture is also determined. The model set up for hypertuning is called a *hypermodel*.

Define a hypermodel through two approaches:

* By using a model builder function
* By subclassing the `HyperModel` class of the Keras Tuner API

In [6]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`. [Hyperband](https://arxiv.org/pdf/1603.06560.pdf) tuner is used in this tutorial.

To instantiate the Hyperband tuner, specify the hypermodel and the `objective` to optimize and the maximum number of epochs to train (`max_epochs`).

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

INFO:tensorflow:Reloading Oracle from existing project my_dir\intro_to_kt\oracle.json
INFO:tensorflow:Reloading Tuner from my_dir\intro_to_kt\tuner0.json


The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + log<sub>`factor`</sub>(`max_epochs`) and rounding it up to the nearest integer.

Create a callback to stop training early after reaching a certain value for the validation loss.

In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [9]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 10 Complete [00h 00m 55s]
val_accuracy: 0.8882499933242798

Best val_accuracy So Far: 0.8882499933242798
Total elapsed time: 00h 06m 25s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 256 and the optimal learning rate for the optimizer
is 0.001.



## Train the model

Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [10]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 7s 3ms/step - loss: 0.5021 - accuracy: 0.8242 - val_loss: 0.4242 - val_accuracy: 0.8500
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3735 - accuracy: 0.8638 - val_loss: 0.3483 - val_accuracy: 0.8748
Epoch 3/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3341 - accuracy: 0.8774 - val_loss: 0.3343 - val_accuracy: 0.8772
Epoch 4/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3064 - accuracy: 0.8868 - val_loss: 0.3290 - val_accuracy: 0.8811
Epoch 5/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2900 - accuracy: 0.8922 - val_loss: 0.3541 - val_accuracy: 0.8742
Epoch 6/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2745 - accuracy: 0.8995 - val_loss: 0.3128 - val_accuracy: 0.8871
Epoch 7/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2637 - accuracy: 0.9023 - val_loss: 0.3385 - val_accuracy:

Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

In [11]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/44
1500/1500 [==============================] - 7s 4ms/step - loss: 0.5090 - accuracy: 0.8203 - val_loss: 0.4184 - val_accuracy: 0.8499
Epoch 2/44
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3775 - accuracy: 0.8652 - val_loss: 0.3936 - val_accuracy: 0.8552
Epoch 3/44
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3368 - accuracy: 0.8775 - val_loss: 0.3588 - val_accuracy: 0.8694
Epoch 4/44
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3118 - accuracy: 0.8845 - val_loss: 0.3420 - val_accuracy: 0.8781
Epoch 5/44
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2939 - accuracy: 0.8910 - val_loss: 0.3495 - val_accuracy: 0.8734
Epoch 6/44
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2768 - accuracy: 0.8961 - val_loss: 0.3098 - val_accuracy: 0.8892
Epoch 7/44
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2633 - accuracy: 0.9015 - val_loss: 0.3111 - val_accuracy:

To finish this tutorial, evaluate the hypermodel on the test data.

In [12]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 2s 6ms/step - loss: 0.5062 - accuracy: 0.8881
[test loss, test accuracy]: [0.5062196254730225, 0.8881000280380249]


The `my_dir/intro_to_kt` directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional `overwrite=True` argument while instantiating the tuner.